## Using Selenium to Automate the process of downloading files and preparing them in the desired format

In [2]:
#importing the required libraries
import selenium as se
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.edge.service import Service as EdgeService

In [3]:
li = ['CIPLA','DIVISLAB','DRREDDY','SUNPHARMA','LUPIN']

In [7]:
#a function for downloading the data for a index and into a specified folder 
def data_downloader(location,comp_list): 
    from time import sleep
    options = webdriver.EdgeOptions()
    '''Asking for a custom folder location'''
    pref = {"download.default_directory" : location}
    options.add_experimental_option("prefs",pref)
    driver = webdriver.Edge(options=options)
    driver.get('https://www.screener.in/login/?next=/user/company/export/6596470/')
    sleep(2)
    username = driver.find_element(By.XPATH,("//input[@name='username']"))
    password = driver.find_element(By.XPATH,("//input[@name='password']"))
    '''Input your email ID and Password here (Screener)'''
    username.send_keys('jubinjoseph917@gmail.com')
    password.send_keys('cfcplayerno.1')
    sleep(2)
    login = driver.find_element(By.XPATH,('//button[@type="submit"]')) 
    login.click()
    sleep(2)
    #downloading the data by opening new tabs and editing the links and closing the tabs after data is downloaded
    driver.execute_script("window.open('');")
    '''The comp list is a list of ticker names'''
    for comp in comp_list: 
        new_url = f"https://www.screener.in/company/{comp}/consolidated/"
        sleep(3)
        driver.switch_to.window(driver.window_handles[1])
        driver.get(new_url)
        sleep(3)
        try:
            download = driver.find_element(By.XPATH,("//button[@aria-label='Export to Excel']"))
            download.click()
        except:
            print(f"Data not Found for {comp}")            
            continue
        sleep(5)
    driver.close()

In [8]:
#SBILIFE AND NESTLE DATA DOWNLOADED SEPERATELY ALSO JIOFINANCIAL SERVICES WAS NEWLY LISTED
data_downloader(r"C:\Users\jubin\OneDrive\Desktop\Analytics EXPO\Competitors Advanced\Financial Statements",li)

# Data Fetching and Fundamental Analyis

In [9]:
#importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os as os
%matplotlib inline

## Data Manipulation and Editing

In [10]:
#definition to edit balance sheets obtained from screener and uplaoding it back
def bs_setter(dataframe): 
    #dropping null values and Transposing the frame
    temp = dataframe.dropna().T
    #setting the column names as one of the row with row names
    temp.columns = temp.iloc[0,:]
    #changing the index to the date values and filtering data
    temp = temp.iloc[1:,:].set_index('Report Date')
    for col in temp.columns:
        temp[col]=temp[col].astype('float')
    temp=temp.round(2)
    return temp

In [11]:
#definition to edit balance sheets obtained from screener and uplaoding it back
def pl_setter(dataframe): 
    #dropping null values and Transposing the frame
    temp = dataframe.dropna().T
    #setting the column names as one of the row with row names
    temp.columns = temp.iloc[0,:]
    #changing the index to the date values and filtering data
    temp = temp.iloc[1:11,:].set_index('Report Date')
    for col in temp.columns:
        temp[col]=temp[col].astype('float')
    temp=temp.round(2)
    return temp

In [12]:
#definition to edit cashflow obtained from screener and uplaoding it back
def cfs_setter_price(dataframe): 
    #dropping null values and Transposing the frame
    temp = dataframe.dropna().T
    #setting the column names as one of the row with row names
    temp.columns = temp.iloc[0,:]
    #changing the index to the date values and filtering data
    temp = temp.iloc[1:11,:].set_index('Report Date')
    for col in temp.columns:
        temp[col]=temp[col].astype('float')
    temp=temp.round(2)
    return temp

In [13]:
#function to read all the excel files with financial data and combine them
def data_reader(folder_path):
    # Iterate through all files in the folder
    all_data={}
    x=0
    all_balance_sheet=[]
    all_profit_and_loss=[]
    for filename in os.listdir(folder_path):
        if filename.endswith('.xlsx'):  # Assuming you're working with .xlsx files
            file_path = os.path.join(folder_path, filename)
            # Load the Excel file
            data = pd.read_excel(file_path,sheet_name='Data Sheet')
            x=x+1
            #slicing the values for the balancesheet (formatting too)
            balance_sheet = bs_setter(data.loc[54:70,:].drop(data.columns[[1]],axis=1))
            balance_sheet.columns.name = filename[:-5]
            #slicing the values for the incomestatement (formatting too)
            profit_loss = pl_setter(data.loc[14:29,:].drop(data.columns[[1]],axis=1))
            profit_loss.columns.name = filename[:-5]
            #slicing values for cashflow statemnent (formatting too)
            cash_flow_statement = cfs_setter_price(data.iloc[78:].drop(data.columns[[1]],axis=1))
            cash_flow_statement.columns.name = filename[:-5]
            #appending the p&l frames to a list
            data_1=pd.merge(balance_sheet,profit_loss,left_index=True,right_index=True,how='left')
            data_fin = pd.merge(data_1,cash_flow_statement,left_index=True,right_index=True,how='left')
            data_fin = data_fin.sort_index(ascending=False)
            all_data[filename[:-5]] = data_fin
        else:
            print(f"Issues at file index {x}")
    return all_data

In [45]:
data=data_reader(r"C:\Users\jubin\OneDrive\Desktop\Analytics EXPO\Competitors Advanced\Financial Statements")

## Formulas for ratio computaion

In [46]:
#creating the required ratios   
def ratios_computer_req(df):
    #Debt to Equity
    #EBIT
    df['EBIT'] = df['Profit before tax'] + df['Interest']
    #D/E Ratio
    df['Debt_to_Equity'] = df['Borrowings'] / (df['Equity Share Capital'] + df['Reserves'])
    #Debtor Days
    df['Debtor_Days'] = df['Receivables'] / (df['Sales']/365)
    #Return on equity
    df['ROE'] = df['Net profit'] / (df['Equity Share Capital'] + df ['Reserves'])
    #Return on Assets
    df['ROA'] = df['Net profit'] / (df['Net Block'] + df['Capital Work in Progress'] + df['Investments'] + df['Other Assets'])
    #Return on capital employed
    df['ROCE'] = df['EBIT'] * 2 / ((df['Equity Share Capital']+df['Equity Share Capital'].shift(-1)) + (df['Borrowings']+df['Borrowings'].shift(-1)) + (df['Reserves'] +df['Reserves'].shift(-1)))                         
    #Net Profit Margin
    df['Net_Profit_Margin'] = df['Net profit'] / df['Sales'] * 100
    #COGS
    df['COGS'] = df['Raw Material Cost'] + df['Power and Fuel'] + df['Other Mfr. Exp'] + df['Employee Cost']
    #Gross margin
    df['Gross_Margin'] = (df['Sales'] - df['COGS']) / df['Sales'] 
    #Asset Turnover Ratio
    df['Asset_Turnover_Ratio']=(df['Sales'] * 2) / ((df['Net Block'] + df['Capital Work in Progress'] + df['Investments'] + df['Other Assets']) + (df['Net Block'].shift(-1) + df['Capital Work in Progress'].shift(-1) + df['Investments'].shift(-1) + df['Other Assets'].shift(-1)))
    #Inventory days
    df['Inventory_Days'] = ((df['Inventory'] + df['Inventory'].shift(-1))/2) / df['COGS'] * 365 

In [47]:
#computing ratios for all the companies
for x in data:
    ratios_computer_req(data[x])

In [48]:
#creating a comprehensive dataframe
all_data = pd.DataFrame()
for x in data:
    data[x]['Comp_Name'] = x
    all_data = pd.concat([data[x],all_data],axis=0)
all_data.sort_index(ascending=True).to_csv(r'C:\Users\jubin\OneDrive\Desktop\Analytics EXPO\Competitors Advanced\All_Data\All_Data.csv')

In [8]:
#function to compute the desired ratios on the dataframes stored in the dictionary
def ratios_computer(df):
    
    #Computing the profitability ratios
    #EBITDA
    df['EBITDA'] = df['Profit before tax'] + df['Interest'] + df['Depreciation']
    
    #EBIT
    df['EBIT'] = df['Profit before tax'] + df['Interest']
    
    #Market Cap 
    df['Market Cap'] = df['Equity Share Capital'] * df['PRICE:'] / 10000000
    
    #EV
    df['EV'] = df['Market Cap'] + df['Borrowings'] - df['Cash & Bank']
    
    
    #Valuation Multiples
    #EV/EBITDA
    df['EV/EBITDA'] = df['EV'] / df['EBITDA']
    
    #EV/EBIT
    df['EV/EBIT'] = df['EV'] / df['EBIT']
    
    #EV/Sales
    df['EV/Sales'] = df['EV'] / df['Sales']
    
    #EPS 
    df['EPS'] = df['Net profit'] / df['Equity Share Capital']
    
    #P/E Ratio
    df['P/E'] = df['PRICE:'] / df['EPS'] 
    
    #Profitability Ratios
    #Return on Equity
    df['ROE'] = df['Net profit'] / (df['Equity Share Capital'] + df ['Reserves'])
    
    #Return on Assets
    df['ROA'] = df['Net profit'] / (df['Net Block'] + df['Capital Work in Progress'] + df['Investments'] + df['Other Assets'])
    
    #Return on capital employed
    df['ROCE'] = df['EBIT'] * 2 / ((df['Equity Share Capital']+df['Equity Share Capital'].shift(-1)) + (df['Borrowings']+df['Borrowings'].shift(-1)) + (df['Reserves'] +df['Reserves'].shift(-1)))                         
    
    #Solvency Ratios
    #Interest Coverage Ratio
    df['Interest_Cov_Rat'] = df['EBIT'] / df['Interest']
    
    #Debt to Equity
    df['Debt_to_Equity'] = df['Borrowings'] / (df['Equity Share Capital'] + df['Reserves'])
    
    #Growth Ratios
    #Sales 
    df['Yearly_Sales_Growth'] = (df['Sales'] / df['Sales'].shift(-1)) - 1 
    df['Sales_3yr_cagr'] = (df['Sales'] / df['Sales'].shift(-3))**(1/3) - 1
    df['Sales_5yr_cagr'] = (df['Sales'] / df['Sales'].shift(-5))**(1/5) - 1
    #Net Profit
    df['Yearly_Profit_Growth'] = (df['Net profit'] / df['Net profit'].shift(-1)) - 1 
    df['Profit_3yr_cagr'] = (df['Net profit'] / df['Net profit'].shift(-3))**(1/3) - 1
    df['Profit_5yr_cagr'] = (df['Net profit'] / df['Net profit'].shift(-5))**(1/5) - 1
    
    #EBITDA Growth
    df['Yearly_EBITDA_Growth'] = (df['EBITDA'] / df['EBITDA'].shift(-1)) - 1 
    df['EBIITDA_3yr_cagr'] = (df['EBITDA'] / df['EBITDA'].shift(-3))**(1/3) - 1
    df['EBITDA_5yr_cagr'] = (df['EBITDA'] / df['EBITDA'].shift(-5))**(1/5) - 1
    
    #EBIT Growth
    df['Yearly_EBIT_Growth'] = (df['EBIT'] / df['EBIT'].shift(-1)) - 1 
    df['EBIT_3yr_cagr'] = (df['EBIT'] / df['EBIT'].shift(-3))**(1/3) - 1
    df['EBIT_5yr_cagr'] = (df['EBIT'] / df['EBIT'].shift(-5))**(1/5) - 1
    
    #EPS Growth
    df['Yearly_EPS_Growth'] = (df['EPS'] / df['EPS'].shift(-1)) - 1 
    df['EPS_3yr_cagr'] = (df['EPS'] / df['EPS'].shift(-3))**(1/3) - 1
    df['EPS_5yr_cagr'] = (df['EPS'] / df['EPS'].shift(-5))**(1/5) - 1

In [ ]:
#Liquidity Ratios have not been computed due to absence of line items
#Any other ratios that you need Kindly let me know
#Some more cleaning can be done which we will get to towards the end